In [1]:
!python --version
!pyspark --version

Python 3.7.12
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 2.4.0
      /_/
                        
Using Scala version 2.11.12, OpenJDK 64-Bit Server VM, 1.8.0_312
Branch 
Compiled by user  on 2018-10-29T06:22:05Z
Revision 
Url 
Type --help for more information.


In [2]:
import sagemaker_pyspark
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

classpath = ":".join(sagemaker_pyspark.classpath_jars())
print(classpath)
spark = SparkSession.builder \
    .config("spark.driver.extraClassPath", classpath) \
    .config("spark.executor.extraClassPath", classpath) \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-2.4_2.11:1.2.0-incubating,org.apache.sedona:sedona-viz-2.4_2.11:1.2.0-incubating,org.datasyslab:geotools-wrapper:1.1.0-25.2,') \
    .getOrCreate()
SedonaRegistrator.registerAll(spark)
# suppresses a bunch of AbortableS3Stream warnings
spark.sparkContext.setLogLevel("ERROR")

/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-core-1.11.835.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-kms-1.11.835.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-s3-1.11.835.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemaker-1.11.835.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemakerruntime-1.11.835.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/aws-java-sdk-sts-1.11.835.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/hadoop-annotations-2.8.1.jar:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/sagemaker_pyspark/jars/hadoop-auth-2.8.1.ja

Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
:: loading settings :: url = jar:file:/home/ec2-user/micromamba/envs/spark_2_4_0/lib/python3.7/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-2.4_2.11 added as a dependency
org.apache.sedona#sedona-viz-2.4_2.11 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-66c584c9-3937-4ea7-a8cb-157f13e27de7;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-2.4_2.11;1.2.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-core;2.12.2 in central

2022-07-17 19:48:35 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-07-17 19:48:36 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
%%time
# Est. 50 sec
# load in the craigslist data
craigslist = spark.read.parquet("s3a://tcceval-data/craigslist/")
craigslist.createOrReplaceTempView("craigslist")

CPU times: user 8.95 ms, sys: 435 µs, total: 9.39 ms
Wall time: 44.9 s


In [4]:
craigslist.printSchema()
print(craigslist.rdd.getNumPartitions())

root
 |-- pid: string (nullable = true)
 |-- repostid: string (nullable = true)
 |-- dt: timestamp (nullable = true)
 |-- url: string (nullable = true)
 |-- title: string (nullable = true)
 |-- price: double (nullable = true)
 |-- neighb: string (nullable = true)
 |-- beds: long (nullable = true)
 |-- sqft: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- accuracy: double (nullable = true)
 |-- address: string (nullable = true)
 |-- posttext: string (nullable = true)
 |-- bedsqft: string (nullable = true)
 |-- subdomain: string (nullable = true)
 |-- subdir: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)

550


In [7]:
craigslist.repartition(12).write.parquet("s3a://tcceval-data/craigslist-squashed/", compression="gzip")